In [1]:
import os

print(os.getcwd())
print(os.listdir())

/teamspace/studios/this_studio/ChessOps
['.git', '.gitignore', 'README.md', 'add_bos.py', 'backend', 'backup', 'checkDuplicates.py', 'create_tokens.py', 'data', 'dataPreProcessing.ipynb', 'env.yaml', 'finetune.ipynb', 'finetune.py', 'inference.ipynb', 'models', 'output', 'pgn_to_xlan.py', 'pipeline.ipynb', 'pipeline.png', 'pipeline.yaml', 'remove_lines.py', 'requirements.txt', 'src', 'statistics', 'streamlit', 'wandb']


In [2]:
token_path = "./src/tokenizer/xlanplus_tokens.json"  # tokenizer needs this to know which tokens to use

In [3]:
from transformers import AutoModelForCausalLM, GPT2LMHeadModel

model_id = "Leon-LLM/Leon-Chess-350k-Plus"
# peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_carlsen_5E_0.0001LR"
# peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_kasparov_5E_0.0001LR"
# peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_kasparov_10E_0.0001LR"
peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_carlsen_10E_0.0001LR"
model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
input_string = ""
number_of_games = 1000
plies = 125  # number of plies (half-moves) to predict => 125 seems to be the maximum
number_of_tokens_per_ply = 4
temperature = 0.2  # higher is more random, lower is more deterministic
max_batch_size = 16  # maximum number of predictions to make in parallel (higher is faster but requires more memory)

In [5]:
from src.generate_prediction import generate_batch_predictions

output_batch, predicted_tokens_batch, input_tokens_batch = generate_batch_predictions(
    inputs=[input_string]*number_of_games,
    num_tokens_to_generate=plies * number_of_tokens_per_ply,
    model=model,
    token_path=token_path,
    temperature=temperature,
    max_batch_size=max_batch_size,
)

print("output_batch: ", output_batch)
for game in output_batch:
    print(f"len(game.split(' ')): {len(game.split(' '))}")

output_batch:  ['Pe2e4- Pc7c5- Pd2d4- Pc5d4x Ng1f3- Nb8c6- Nb1c3- Pd4c3x Pb2c3x Pe7e6- Bf1b5- Pa7a6- Bb5a4- Pb7b5- Ba4b3- Ng8f6- Ke1g1- Bf8e7- Rf1e1- Ke8g8- Pa2a4- Pb5b4- Pc3b4x Nc6b4x Bc1a3- Nb4c6- Pc2c4- Bc8b7- Ba3e7x Qd8e7x Pc4c5- Nc6a5- Bb3a2- Na5c6- Pa4a5- Nc6a5x Qd1a4- Na5c6- Ra1b1- Bb7c6- Qa4b3- Nc6a5- Qb3b4- Na5c6- Qb4b2- Rf8b8- Qb2c3- Rb8b1x Re1b1x Qe7c5x Qc3c5x Na5c6- Qc5b6- Nc6e7- Qb6c5- Ra8a2x Rb1b8+ Ne7c8- Qc5c7- Ra2a1+ Qc7c8$ Bc6e8- Qc8e8$ Nf6e8x Rb8e8$ Kg8f6- Re8b8- Ra1c1+ Nf3e1- Rc1e1$ Kg1f1- Re1e4x Pg2g3- Re4c4- Kf1g2- Rc4c2- Rb8b6- Rc2c3- Rb6a6x Rc3c2- Ra6a7- Rc2c3- Ra7d7x Rc3c2- Rd7d8- Rc2c3- Rd8d4- Rc3c2- Rd4b4- Rc2c3- Rb4e4- Rc3c6- Re4b4- Rc6c5- Rb4e4- Rc5d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4- Rd5d6- Rb4e4- Rd6d5- Re4b4-', 'Pe2e4- Pe7e5- Pd2d4- Pe5d4x Pc2c3- Nb8c6- Ng1f3- Pd4c3x Nb1c3x Ng8f6- Bf1b5- Bf8b4- Bc1g5- Ph7h6- Bg5f6x 

In [6]:
from src.validation.validate_sequence import evaluate_sequence

result = evaluate_sequence(output_batch, predicted_tokens_batch, debug=False, notation="xLANplus")

# averages of 'number of moves until error'
sum_moves = 0
for game in result:
    sum_moves += game[1]
average_moves = sum_moves / len(result)
print(f"Average number of moves until error: {average_moves}")

Average number of moves until error: 56.134


In [7]:
model_name = peft_model_id.split("/")[-1]
model_name

'Leon-Chess-350k-Plus_LoRA_fcarlsen_10E_0.0001LR'

In [8]:
# write data to file
# csv should have the following columns:
# 'Game String' (full game generated by model) = result[i][0]
# 'Moves Until Error' = result[i][1]
# 'Error Type' = result[i][2]
# 'First Illegal Move' (game string up to the first illegal move) = result[i][3]
# 'Predicted Tokens' (full game in tokenized form) = predicted_tokens_batch[i]

import pandas as pd

# output_path = f"./statistics/statistical_evaluations{model_name}_sequences_{number_of_games}x{plies}.csv"
output_path = f"./statistics/statistical_evaluations_{model_name}_sequences_{number_of_games}x{plies}.csv"

data = {
    'Game String': [game[0] for game in result],
    'Moves Until Error': [game[1] for game in result],
    'Error Type': [game[2] for game in result],
    'First Illegal Move': [game[3] for game in result],
    'Predicted Tokens': predicted_tokens_batch
}

# for every game in 'First Illegal Move' (game[3]), convert the sequence to PGN

df = pd.DataFrame(data)
df.to_csv(output_path, index=False)

print(f"Data written to {output_path}")

Data written to ./statistics/statistical_evaluations_Leon-Chess-350k-Plus_LoRA_fcarlsen_10E_0.0001LR_sequences_1000x125.csv
